In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import heapq
from queue import PriorityQueue
import copy

In [80]:
from tqdm import tqdm

In [81]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose

# GRAPH STUFF

In [82]:
mapa = pd.read_csv("darkstore_map.csv", sep=";")

In [83]:
mapa.head()

,SECTION,LEVEL,LAGERID
0,1,1,1
1,1,2,2
2,1,3,3
3,2,1,4
4,2,2,5


In [84]:
cheque =  pd.read_csv("cheques_public.csv", sep=";")

In [85]:
cheque.head()

,CHEQUEID,LAGERID,KOLVO
0,22311,1,1.0
1,22311,2,1.0
2,22311,3,1.0
3,22311,4,1.0
4,22311,5,1.0


In [86]:
section_to_node_dct = {0:0, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:28, 8:27, 9:26, 10:25, 11:24, 12:23, 13:22, 14:21, 15:28, 16:27, 17:26, 18:25, 19:24, 20:23, 21:22, 22:21, 23:7, 24:8, 25:9, 26:10, 27:11, 28:12, 29:13, 30:14, 31:7, 32:8, 33:9, 34:10, 35:11, 36:12, 37:13, 38:14, 39:20, 40:19, 41:18, 42:17, 43:16, 44:15}
node_to_section_dct = dict((v,k) for k,v in section_to_node_dct.items())

def section_to_node(section):
    return section_to_node_dct[section]

def node_to_section(node):
    return node_to_section_dct[node]

In [87]:
graph_one_side = {(0, 1): 8,
        (0, 20): 18,
        (1, 2): 2,
        (2, 3): 2,
        (3, 4): 2,
        (4, 5): 2,
        (5, 6): 2,
         
        (5, 7): 2,
        (7, 8): 2,
        (8, 9): 2,
        (9, 10): 2,
        (10, 11): 2,
        (11, 12): 2,
        (12, 13): 2,
        (13, 14): 2,
         
        (14, 16): 2,
        (16, 15): 2,
        (16, 17): 2,
        (17, 18): 2,
        (18, 19): 2,
         
        (19, 20): 2,
        (19, 21): 2,
        (21, 22): 2,
        (22, 23): 2,
        (23, 24): 2,
        (24, 25): 2,
        (25, 26): 2,
        (26, 27): 2,
        (27, 28): 2,
        (28, 2): 2}

In [88]:
graph_another_side = {(k[1], k[0]):v for k,v in graph_one_side.items()}

In [89]:
graph = {**graph_one_side, **graph_another_side}

In [90]:
class Graph:
    def __init__(self, num_of_vertices):
        self.v = num_of_vertices
        self.edges = [[-1 for i in range(num_of_vertices)] for j in range(num_of_vertices)]
        self.visited = []
        
    def add_edge(self, u, v, weight):
        self.edges[u][v] = weight
        self.edges[v][u] = weight
        
    def dijkstra(self, start_vertex):
        D = {v:float('inf') for v in range(self.v)}
        D[start_vertex] = 0

        pq = PriorityQueue()
        pq.put((0, start_vertex))

        while not pq.empty():
            (dist, current_vertex) = pq.get()
            self.visited.append(current_vertex)

            for neighbor in range(self.v):
                if self.edges[current_vertex][neighbor] != -1:
                    distance = self.edges[current_vertex][neighbor]
                    if neighbor not in self.visited:
                        old_cost = D[neighbor]
                        new_cost = D[current_vertex] + distance
                        if new_cost < old_cost:
                            pq.put((new_cost, neighbor))
                            D[neighbor] = new_cost
        return D

In [91]:
g = Graph(29)

for key, value in graph_one_side.items():
    g.add_edge(key[0], key[1], value)

In [92]:
D = g.dijkstra(0)

for vertex in range(len(D)):
    print("Distance from vertex 0 to vertex", vertex, "is", D[vertex])

Distance from vertex 0 to vertex 0 is 0
Distance from vertex 0 to vertex 1 is 8
Distance from vertex 0 to vertex 2 is 10
Distance from vertex 0 to vertex 3 is 12
Distance from vertex 0 to vertex 4 is 14
Distance from vertex 0 to vertex 5 is 16
Distance from vertex 0 to vertex 6 is 18
Distance from vertex 0 to vertex 7 is 18
Distance from vertex 0 to vertex 8 is 20
Distance from vertex 0 to vertex 9 is 22
Distance from vertex 0 to vertex 10 is 24
Distance from vertex 0 to vertex 11 is 26
Distance from vertex 0 to vertex 12 is 28
Distance from vertex 0 to vertex 13 is 30
Distance from vertex 0 to vertex 14 is 28
Distance from vertex 0 to vertex 15 is 28
Distance from vertex 0 to vertex 16 is 26
Distance from vertex 0 to vertex 17 is 24
Distance from vertex 0 to vertex 18 is 22
Distance from vertex 0 to vertex 19 is 20
Distance from vertex 0 to vertex 20 is 18
Distance from vertex 0 to vertex 21 is 22
Distance from vertex 0 to vertex 22 is 24
Distance from vertex 0 to vertex 23 is 22
Dist

In [93]:
def get_optimal_distance_between_nodes(node_a, node_b, graph):
    D = graph.dijkstra(node_a)
    # for vertex in range(len(D)):
    #     print("Distance from vertex 0 to vertex", vertex, "is", D[vertex])
        
    return D[node_b]

def get_optimal_distance_between_sections(sec_a, sec_b, graph):
    node_a = section_to_node(sec_a)
    node_b = section_to_node(sec_b)
    
    # print(node_a, node_b)
    
    return get_optimal_distance_between_nodes(node_a, node_b, graph)

In [94]:
g = Graph(29)

for key, value in graph_one_side.items():
    g.add_edge(key[0], key[1], value)

In [95]:
get_optimal_distance_between_sections(12, 24, copy.deepcopy(g))

22

In [96]:
def unique_custom(x):
    to_del = []
    dict_freq = {}
    for c,i in enumerate(x):
        freq = dict_freq.get(i,0)
        if freq==1:
            to_del.append(c)
        else:
            dict_freq.update({i:freq+1})
            
    return [i for c,i in enumerate(x) if not c in to_del]

## Creation of one function

In [19]:
cheque[cheque['CHEQUEID']==22311]

,CHEQUEID,LAGERID,KOLVO
0,22311,1,1.0
1,22311,2,1.0
2,22311,3,1.0
3,22311,4,1.0
4,22311,5,1.0
5,22311,6,1.0
6,22311,7,1.0
7,22311,8,1.0
8,22311,9,1.0
9,22311,10,1.0


In [20]:
cheque = pd.merge(cheque, mapa, on='LAGERID', how='left')

In [21]:
cheque['TIME_SPENT_GETTING'] = cheque['KOLVO']*cheque['LEVEL']

In [22]:
aggregated = cheque.groupby('CHEQUEID', as_index=False).agg({'SECTION':list,"TIME_SPENT_GETTING":sum })

In [23]:
dict_section = aggregated.set_index('CHEQUEID')['SECTION'].to_dict()
dict_time_getting = aggregated.set_index('CHEQUEID')['TIME_SPENT_GETTING'].to_dict()

In [24]:
overall_dict = {}
for k,v in dict_section.items():
    time = dict_time_getting.get(k)
    overall_dict.update({k:(v, time)})

# Faster function

In [97]:
def compute_distances(sections,g):
    sections=unique_custom(sections)
    mapping = dict([(i,c+1) for c,i in enumerate(sections)])
    sections.insert(0,0)

    distances = []
    for i in range(len(sections)):
        for j in range(i+1,len(sections)):
            distances.append((sections[i],sections[j],get_optimal_distance_between_sections(sections[i], sections[j], copy.deepcopy(g))))
    distances = [(mapping.get(i[0],0),mapping.get(i[1],0),i[2]) for i in distances]
    to_exclude = 0
    new_distances = []
    for i in distances:
        if i[-1]==0:
            to_exclude+=1
            new_distances.append([i[0],i[1],1])
        else:
            new_distances.append(i)
    distances = new_distances
    return (distances,to_exclude, len(sections))


In [98]:
def pre_compute_distances_with_all(sections,all_distances):
    sections=unique_custom(sections)
    mapping = dict([(i,c+1) for c,i in enumerate(sections)])
    sections.insert(0,0)
    distances = []
    for i in range(len(sections)):
        for j in range(i+1,len(sections)):
            idx1 = (sections[i],sections[j])
            idx2 = (sections[j],sections[i])
            distance = all_distances.get(idx1, all_distances.get(idx2))            
            distances.append((sections[i],sections[j],distance))
    distances = [(mapping.get(i[0],0),mapping.get(i[1],0),i[2]) for i in distances]
    to_exclude = 0
    new_distances = []
    for i in distances:
        if i[-1]==0:
            to_exclude+=1
            new_distances.append([i[0],i[1],1])
        else:
            new_distances.append(i)
    distances = new_distances
    return (distances,to_exclude, len(sections))


In [99]:
def compute_distances_all(num_all, g):
    sections=[i for i in range(0,num_all+1)]
    distances = {}
    for i in range(len(sections)):
        for j in range(i+1,len(sections)):
            distances.update({(sections[i],sections[j]):get_optimal_distance_between_sections(sections[i], sections[j], copy.deepcopy(g))})

    return distances


In [100]:
def find_best_path_dist(distances, len_sections, evaluate=False):
    fitness_dists = mlrose.TravellingSales(distances = distances)
    problem_fit = mlrose.TSPOpt(length = len_sections, fitness_fn = fitness_dists, maximize=False)
    if not evaluate:
        best_state, best_fitness = mlrose.random_hill_climb(problem_fit, random_state = 0, max_iters=4, restarts=1, max_attempts=12)
    else:
        best_state, best_fitness = mlrose.random_hill_climb(problem_fit, random_state = 0, max_iters=100, restarts=5, max_attempts=30)
    return best_fitness

In [101]:
def loss(distances, to_exclude,len_sections, time_get_goods, evaluate=False):
    #finding best path dist
    best_path_dist = find_best_path_dist(distances,len_sections, evaluate)
    #finding time to spend on taking goods
    loss_ = time_get_goods+best_path_dist-to_exclude
    return loss_
    

In [71]:
objective = 0
for v in tqdm(dict_with_distances.values()):
    distance,to_exclude, len_section = v[0]
    time_get_goods = v[1]
    objective+=loss(distance,to_exclude, len_section, time_get_goods)

100%|███████████████████████████████████████| 5822/5822 [06:07<00:00, 15.85it/s]


In [72]:
objective

414136.0

In [102]:
all_distances = compute_distances_all(44, g)

In [103]:
all_distances.get((0,1))

8

# Testing submission

In [55]:
test_sub = pd.read_csv('test_sub_new.csv', sep=';')

In [56]:
new_cheques = pd.read_csv('cheques_public.csv', sep=';')

In [57]:
test_sub

,SECTION,LEVEL,LAGERID
0,1,1,15
1,1,2,29
2,1,3,3
3,2,1,7
4,2,2,57
...,...,...,...
127,43,2,124
128,43,3,93
129,44,1,8
130,44,2,120


In [58]:
new_cheques = new_cheques.merge(test_sub, on='LAGERID', how='left')

In [59]:
new_cheques['TIME_SPENT_GETTING'] = new_cheques['KOLVO']*new_cheques['LEVEL']

In [60]:
aggregated = new_cheques.groupby('CHEQUEID', as_index=False).agg({'SECTION':list,"TIME_SPENT_GETTING":sum })

In [61]:
dict_section = aggregated.set_index('CHEQUEID')['SECTION'].to_dict()
dict_time_getting = aggregated.set_index('CHEQUEID')['TIME_SPENT_GETTING'].to_dict()

In [62]:
overall_dict = {}
for k,v in dict_section.items():
    time = dict_time_getting.get(k)
    overall_dict.update({k:(v, time)})

In [136]:
dict_with_distances = {}
for k,v in tqdm(overall_dict.items()):
    distances = compute_distances(v[0], copy.deepcopy(g))
    dict_with_distances.update({k:(distances, v[1])})

100%|███████████████████████████████████████| 5822/5822 [01:22<00:00, 70.54it/s]


In [122]:
objective = 0
for v in tqdm(dict_with_distances.values()):
    distance,to_exclude, len_section = v[0]
    time_get_goods = v[1]
    objective+=loss(distance,to_exclude, len_section, time_get_goods)

100%|███████████████████████████████████████| 5822/5822 [01:44<00:00, 55.65it/s]


# Optimization class

In [104]:
class Optimization:
    def __init__(self, cheque_df, test_sub, graph, all_distances, evaluate=False):
        self.evaluate = evaluate
        self.cheque_df = cheque_df
        self.test_sub = test_sub
        self.graph = graph
        self.all_distances = all_distances
        self.evaluate = evaluate
    
    def create_needed_data(self, x):
        self.test_sub['LAGERID'] = x
        cheque_df = self.cheque_df.merge(self.test_sub, on='LAGERID', how='left')
        cheque_df['TIME_SPENT_GETTING'] = cheque_df['KOLVO']*cheque_df['LEVEL']
        aggregated = cheque_df.groupby('CHEQUEID', as_index=False).agg({'SECTION':list,"TIME_SPENT_GETTING":sum })
        dict_section = aggregated.set_index('CHEQUEID')['SECTION'].to_dict()
        dict_time_getting = aggregated.set_index('CHEQUEID')['TIME_SPENT_GETTING'].to_dict()
        overall_dict = {}
        for k,v in dict_section.items():
            time = dict_time_getting.get(k)
            overall_dict.update({k:(v, time)})
        return overall_dict


    def pre_compute_distances(self,overall_dict):
        dict_with_distances = {}
        for k,v in tqdm(overall_dict.items()):
            distances = pre_compute_distances_with_all(v[0], self.all_distances)
            dict_with_distances.update({k:(distances, v[1])})
        return dict_with_distances

    def compute_objective(self,dict_with_distances):
        objective = 0
        for v in tqdm(dict_with_distances.values()):
            distance,to_exclude, len_section = v[0]
            time_get_goods = v[1]
            objective+=loss(distance,to_exclude, len_section, time_get_goods, self.evaluate)
        return objective
    
        
        

In [105]:
opt_class = Optimization(cheque_df=pd.read_csv('cheques_public.csv', sep=';'), test_sub=pd.read_csv('darkstore_map.csv', sep=';'), 
            graph=g, all_distances=all_distances, evaluate=True)

In [70]:
# initial = pd.read_csv('sort_in_clust_by_dist_to_zero.csv', sep=',').rename(columns={'SORTED_LAGERID':'LAGERID'})['LAGERID'].values

In [71]:
initial = pd.read_csv('clustered_only_sub.csv', sep=',')['LAGERID'].values

In [72]:
def minimization_function(x, solution_idx):
    global opt_class
    dict_with_distances = opt_class.pre_compute_distances(opt_class.create_needed_data(x))
    return opt_class.compute_objective(dict_with_distances)*(-1)

In [73]:
minimization_function(initial,0)

100%|███████████████████████████████████████| 5822/5822 [02:53<00:00, 33.55it/s]


-398122.0

In [74]:
initial = pd.read_csv('sort_in_clust_by_dist_to_zero.csv', sep=',').rename(columns={'SORTED_LAGERID':'LAGERID'})['LAGERID'].values
minimization_function(initial,0)

100%|███████████████████████████████████████| 5822/5822 [03:05<00:00, 31.43it/s]


-411595.0

In [76]:
initial = pd.read_csv('test_sub_new.csv', sep=';')['LAGERID'].values
minimization_function(initial,0)

100%|███████████████████████████████████████| 5822/5822 [02:29<00:00, 38.99it/s]


-398210.0

In [78]:
initial = pd.read_csv('darkstore_map.csv', sep=';')['LAGERID'].values
minimization_function(initial,0)

100%|███████████████████████████████████████| 5822/5822 [02:46<00:00, 34.97it/s]


-416096.0

In [106]:
initial_ga_cust_mut = [ 44,  26,  50, 126,   2, 116,  42, 114,  27,  69, 123,  53,  98,
        21, 130,  32,  25,  28,  12,  60,  83,  31,  90,  17,  78, 122,
        55,  41, 129,  19,  85,  79, 100,  33,  37,  89,   5,  88,  81,
       119,   4,  18,  15,  29,   3,   7,  57,  14,  30,  51, 107,  20,
       118,  80,  87,  95,  75,  38,  22, 106, 105,  23,  45,   1,  40,
        68,  64,  77,   9,  84,  91,  46,  54,  92,  82,  71, 125,  39,
        10,  99, 121,   6, 108, 131, 128,  59, 110,  58,  52,  24,  62,
       112, 115,  86,  35,  94,  49, 104,  74, 127,  11,  97,  43, 109,
        70,  67,  72,  13,  61,  36, 117,  93,  47, 132,  34,  96, 111,
       113,  63,  16, 103, 102,  56, 101,  65,  73,  76, 124,  48,   8,
       120,  66]

In [107]:
minimization_function(initial_ga_cust_mut,0)

100%|███████████████████████████████████████| 5822/5822 [02:48<00:00, 34.51it/s]


-395241.0

In [108]:
initial_ga_cust_mut2 = [ 83,  98,  21, 130,  32,  25,  51,  12,  60,  31,  90,  17,  78,
       122,  55, 125,   1,  19,  27,  69, 123,  44,  26,  53,  85,  50,
       126,  79, 100,  33,  37,  89,   5,  88,  81, 119,   4,  18,  28,
       107,  43, 118,  15,  29,   3,   7,  57,  14,  30,   2, 116,  42,
       114,  80,  87,  95,  75,  38,  22, 106, 105,  35,  45, 129,  40,
        68,  64,  77,   9,  84,  41,  46,  54,  92,  82,  71,  91,  39,
        10,  99, 121,   6, 108, 131,   8,  59, 110,  16,  52,  24,  62,
       112, 115,  86,  23,  94,  49, 104, 128, 127,  11,  97,  20, 109,
        70,  67,  72,  13,  74,  36, 117,  48,  47, 132,  34,  96, 111,
       113,  61,  63,  58, 103, 102,  56,  76,  65,  73, 101, 124,  93,
       120,  66]

In [109]:
minimization_function(initial_ga_cust_mut2,0)

100%|███████████████████████████████████████| 5822/5822 [02:36<00:00, 37.19it/s]


-394871.0

In [110]:
initial_ga = [  2, 116,  42,  57,  14,  30, 114,  27,  69, 123,  44,  26,  53,
        98,  21, 130,  32,  25,  51,  12,  60,  83,  31,  90,  17,  78,
       122,  55,  41, 129,  19,  94,  50, 126,  79,  70,  33,  37,  89,
         5,  67,  81,  15,  29,   3,   7,  48, 100, 119,   4,  96,  28,
       107,  20, 118,  80,  87,  95,  75, 108,  38,  22, 106, 105,  23,
        45,   1,  40,  68,  64,  77,   9,  84,  91,  46,  54,  92,  82,
        71, 125,  39,  10,  99, 102, 121,   6, 131,  74,  59, 110,  58,
        24,  62, 112, 115,  86,  35,  85,  49, 104, 128, 127,  11,  97,
        43, 109,  88,  72,  13,  61,  36, 117,  52, 120,  47, 132,  34,
        18, 111, 113,  63,  16, 103,  56,  76,  65,  73, 101, 124,  93,
         8,  66]

In [111]:
minimization_function(initial_ga,0)

100%|███████████████████████████████████████| 5822/5822 [02:10<00:00, 44.65it/s]


-393957.0

# Genetic stuff

In [275]:
max(initial), min(initial)

(132, 1)

In [276]:
import pygad

In [381]:
fitness_function = minimization_function
num_generations = 25 # Number of generations.
num_parents_mating = 5 # Number of solutions to be selected as parents in the mating pool.
sol_per_pop = 22 # Number of solutions in the population.
num_genes = 132  

init_range_low = 1
init_range_high = 132

parent_selection_type = "rank" # Type of parent selection.
keep_parents = 5 # Number of parents to keep in the next population. -1 means keep all parents and 0 means keep nothing.

# Parameters of the mutation operation.
mutation_type = "swap" # Type of the mutation operator.
mutation_percent_genes = 10 


In [382]:
population_list = []
gene_space = [i for i in range(1, 132+1)]
for i in range(sol_per_pop):
    nxm_random_num=list(np.random.permutation(gene_space)) 
    population_list.append(nxm_random_num)
initial1 = pd.read_csv('darkstore_map.csv', sep=';')['LAGERID'].values
initial2 = pd.read_csv('test_sub_new.csv', sep=';')['LAGERID'].values
initial3 = pd.read_csv('clustered_only_sub', sep=';')['LAGERID'].values

population_list.append(initial)

In [383]:
def on_generation(ga):
    print("Generation", ga.generations_completed)
    print(ga.population)

In [384]:
def crossover_func(parents, offspring_size, ga_instance):
    offspring = []
    idx = 0
    while len(offspring) != offspring_size[0]:
        childP1 = []
        childP2 = []
        parent1 = parents[idx % parents.shape[0], :].copy()
        parent2 = parents[(idx + 1) % parents.shape[0], :].copy()
        
        geneA = int(random.random() * len(parent1))
        geneB = int(random.random() * len(parent1))
    
        startGene = min(geneA, geneB)
        endGene = max(geneA, geneB)

        for i in range(startGene, endGene):
            childP1.append(parent1[i])
            
        childP2 = [item for item in parent2 if item not in childP1]
        
        offspring.append(np.array(childP1+childP2))

        idx += 1

    return np.array(offspring)

In [385]:
ga_instance = pygad.GA(num_generations=num_generations,
                       num_genes=num_genes,
                       mutation_percent_genes = mutation_percent_genes, 
                       initial_population=population_list,
                       parent_selection_type=parent_selection_type,
                       sol_per_pop = len(population_list),
                       gene_space = gene_space,
                       num_parents_mating=num_parents_mating,
                       mutation_type=mutation_type,
                       fitness_func=fitness_function,
                       crossover_type = crossover_func,
                       gene_type=int,
                       on_generation=on_generation,
                       allow_duplicate_genes=False)

In [ ]:
ga_instance.run()


CHEQUEID    0
LAGERID     0
KOLVO       0
SECTION     0
LEVEL       0
dtype: int64

100%|██████████████████████████████████████| 5822/5822 [00:05<00:00, 972.28it/s]


CHEQUEID    0
LAGERID     0
KOLVO       0
SECTION     0
LEVEL       0
dtype: int64

100%|██████████████████████████████████████| 5822/5822 [00:05<00:00, 982.16it/s]


CHEQUEID    0
LAGERID     0
KOLVO       0
SECTION     0
LEVEL       0
dtype: int64

...

CHEQUEID    0
LAGERID     0
KOLVO       0
SECTION     0
LEVEL       0
dtype: int64

100%|█████████████████████████████████████| 5822/5822 [00:05<00:00, 1097.57it/s]

Generation 16
[[  5   6   7 ... 130 131 132]
 [  5   6  61 ... 130 131 132]
 [ 44  45  46 ... 130 131 132]
 ...
 [ 48  49  50 ... 129 130 131]
 [ 45  46  47 ... 129 130 131]
 [ 87  88  90 ... 130 131 132]]


CHEQUEID    0
LAGERID     0
KOLVO       0
SECTION     0
LEVEL       0
dtype: int64

 47%|█████████████████▎                   | 2716/5822 [00:02<00:02, 1152.95it/s]

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("best_solution: {solution}".format(solution =solution)) 

In [ ]:
solution

In [ ]:
solution_fitness

In [ ]:
minimization_function(solution,0)

In [ ]:
fitnesses = ga_instance.best_solutions_fitness()